In [1]:
import mlflow
from mlflow.tracking import MlflowClient

# Set this to match what your MLflow UI is using (adjust if needed)
mlflow.set_tracking_uri("http://localhost:5000")  # assuming UI is served here

client = MlflowClient()

experiments = client.search_experiments()

for exp in experiments:
    print(f"📁 {exp.experiment_id} — {exp.name}")



/home/mlops/fashion/venv/lib/python3.12/site-packages/mlflow/utils/requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


📁 6 — housing-DecisionTree
📁 5 — Housing-LinearRegression
📁 4 — Iris-Models
📁 3 — FashionMNIST-MLP
📁 2 — FashionMNIST-CNN
📁 0 — Default


In [2]:
import mlflow
from mlflow.tracking import MlflowClient

mlflow.set_tracking_uri("http://localhost:5000")

client = MlflowClient()

# Search for the housing-related experiments
experiments = [exp for exp in mlflow.search_experiments() if "housing" in exp.name.lower()]
all_runs = []

for exp in experiments:
    runs = client.search_runs(exp.experiment_id)
    for run in runs:
        metrics = run.data.metrics
        if "r2_score" in metrics:
            all_runs.append({
                "experiment_name": exp.name,
                "run_id": run.info.run_id,
                "r2_score": metrics["r2_score"],
                "mse": metrics.get("mse", None)
            })

if not all_runs:
    raise ValueError("❌ No valid runs with R² score found!")

# Select best based on R² score
best_run = max(all_runs, key=lambda x: x["r2_score"])
best_run_id = best_run["run_id"]
print(f"✅ Best Housing model: {best_run['experiment_name']} (R²: {best_run['r2_score']:.4f})")



✅ Best Housing model: housing-DecisionTree (R²: 0.5997)


In [ ]:
# Define model name (you can change if needed)
model_name = "HousingBestModel"
artifact_path = "model"  # Assumes the artifact was logged with this name
model_uri = f"runs:/{best_run_id}/{artifact_path}"

# Register the model
model_version = mlflow.register_model(model_uri=model_uri, name=model_name)

# Promote the model to 'Production'
client.transition_model_version_stage(
    name=model_name,
    version=model_version.version,
    stage="Production",
    archive_existing_versions=True
)

print(f"🚀 Model version {model_version.version} promoted to 'Production'")
